# Modèles génériques

In [ ]:
# Librairies à importer pour utiliser JuMP avec le solver GLPK
using JuMP
using GLPK

# Définition de constantes pour le statut de résolution du problème
const OPTIMAL = JuMP.MathOptInterface.OPTIMAL
const INFEASIBLE = JuMP.MathOptInterface.INFEASIBLE
const UNBOUNDED = JuMP.MathOptInterface.DUAL_INFEASIBLE;

Il est possible avec le package JuMP de modéliser des problèmes d'optimisation linéaire sans écrire explicitement les variables et les contraintes.

Reprenons l'exemple des yaourts vu en cours.


>Une entreprise de fabrication de yaourts souhaite produire des yaourts à la fraise. Il lui est possible de fabriquer deux types de yaourts à la fraise : allégé et normal. La fabrication d’un litre de chaque type de yaourt nécessite différentes matières premières : produire un litre de yaourt allégé nécessite 2 kilos de fraises
et 1 litre de lait, alors que la production d’un litre de yaourt normal nécessite 1 kilo de fraises, 2 litres de lait et 1 kilo de sucre. Un litre de yaourt allégé peut être vendu 40 euros alors qu’un litre de yaourt normal peut être vendu 50
euros. 

>Étant donné que l’entreprise possède 800 kilos de fraises, 700 litres de lait et 300 kilos de sucre, combien
de litres de yaourt allégé et normal doit-elle fabriquer pour maximiser son revenu ?

Ce problème linéaire peut se modéliser sous la forme suivante.

In [ ]:
# Création d'un modèle. Ce modèle fera l'interface avec le solveur GLPK
m = Model(with_optimizer(GLPK.Optimizer))

#Variable représentant la quantité en litre de yaourt allégé
@variable(m, xA >= 0)

#Variable représentant la quantité en litre de yaourt "normal"
@variable(m, xN >= 0)

#Maximiser les revenus de la vente
@objective(m, Max, 40xA + 50xN)

#Contrainte liée à la quantité de fraise disponible
@constraint(m, 2xA + xN <= 800)

#Contrainte liée à la quantité de Lait disponible
@constraint(m, xA + 2xN <= 700)

#Contrainte liée à la quantité de sucre disponible
@constraint(m, xN <= 300)

#Affichage du modèle
print(m)

#Résolution du problème d'optimisation linéaire m par le solveur GLPK
optimize!(m)


println("Optimum = ", JuMP.objective_value(m))
println("Fabrication optimale :")
println("Yaourt allégé (en L) = ", JuMP.value(xA))
println("Yaourt \"normal\" = ", JuMP.value(xN))


Ce modèle ne prend en compte que deux types de yaourts. Il est possible de généraliser ce problème à plus de types de yaourts et plus de ressources.
Supposons que l'on puisse fabriquer 12 types de yaourts grâce à 7 ressources disponibles. Les quantités de ressources nécessaires pour fabriquer chaque type de yaourts sont données dans le tableau suivant.


| Yaourts / Ressources  | Fraise         | Framboise  | Mûres      | Vanille    | Lait       | Sucre      | Édulcorant |
| -------------         | -------------: | ---------: | ---------: | ---------: | ---------: | ---------: | ---------: |
| Fraise                | 2 kg/L         | 0 kg/L     | 0 kg/L     | 0 kg/L     | 2 L/L     | 1 kg/L     | 1 kg/L     |
| Fraise allégé         | 2 kg/L         | 0  kg/L    | 0 kg/L     | 0 kg/L     | 1 L/L     | 0 kg/L     | 3 kg/L     |
| Vanille/Fraise        | 2 kg/L         | 0 kg/L     | 0 kg/L     | 2 kg/L     | 5 L/L     | 1 kg/L     | 0 kg/L     |
| Framboise             | 0 kg/L         | 3 kg/L     | 0 kg/L     | 5 kg/L     | 3 L/L     | 4 kg/L     | 0 kg/L     |
| Framboise allégé      | 0 kg/L         | 2 kg/L     | 0 kg/L     | 0 kg/L     | 4 L/L     | 0 kg/L     | 2 kg/L     |
| Vanille/Framboise     | 0 kg/L         | 2  kg/L    | 0 kg/L     | 2 kg/L     | 2 L/L     | 2 kg/L     | 0 kg/L     |
| Mûres                 | 0 kg/L         | 0 kg/L     | 2 kg/L     | 0 kg/L     | 8 L/L     | 2 kg/L     | 0 kg/L     |
| Mûres allégées        | 0 kg/L         | 0 kg/L     | 3 kg/L     | 0 kg/L     | 7 L/L     | 0 kg/L     | 1 kg/L     |
| Vanille/Mûres         | 0 kg/L         | 0 kg/L     | 3 kg/L     | 3 kg/L     | 6 L/L     | 2 kg/L     | 0 kg/L     |
| Fruits rouges         | 1 kg/L         | 2 kg/L     | 2 kg/L     | 1 kg/L     | 3 L/L     | 4 kg/L     | 0 kg/L     |
| Fruits rouges allégé  | 2 kg/L         | 1 kg/L     | 2 kg/L     | 0 kg/L     | 3 L/L     | 2 kg/L     | 1 kg/L     |
| Vanille/Fruits rouges | 1 kg/L         | 1 kg/L     | 1 kg/L     | 3 kg/L     | 3 L/L     | 1 kg/L     | 1 kg/L     |


On connaît également les prix de vente de chaque yaourt et la quantité disponible pour chaque ressource.

| Type de yaourt        | Prix de vente |
| -------------         | ---------:    |
| Fraise                | 50 &euro;/L   |
| Fraise allégé         | 48 &euro;/L   |
| Vanille/Fraise        | 65 &euro;/L   |
| Framboise             | 47 &euro;/L   |
| Framboise allégé      | 52 &euro;/L   |
| Vanille/Framboise     | 46 &euro;/L   |
| Mûres                 | 78 &euro;/L   |
| Mûres allégées        | 75 &euro;/L   |
| Vanille/Mûres         | 83 &euro;/L   |
| Fruits rouges         | 59 &euro;/L   |
| Fruits rouges allégé  | 62 &euro;/L   |
| Vanille/Fruits rouges | 76 &euro;/L   |


| Ressource | Quantité disponible |
| -------------  | ---------:          |
| Fraise         | 800 kg              |
| Framboise      | 200 kg              |
| Mûres          | 150 kg              |
| Vanille        | 80 kg               |
| Lait           | 700 L               |
| Sucre          | 500 kg              |
| Édulcorant     | 150 kg              |


Il est donc possible d'écrire le problème d'optimisation linéaire associé. Cependant, ce dernier aura 12 variables et 7 contraintes. Si l'on ajoute aussi des yaourts qui ne sont pas aux fruits, on peut alors avoir un problème d'optimisation linéaire contenant plusieurs centaines de contraintes et/ou variables. Il n'est donc plus possible d'écrire le problème de manière explicite.


## Formulation générique


Le package JuMP permet d'écrire le modèle sous forme générique. 

### Intérêts

La formulation générique présente deux intérêts :

* Il est possible d'écrire des **problèmes d'optimisation avec beaucoup de variables et/ou de contraintes** puisque l'on n'écrit plus explicitement les variables et les contraintes. Par exemple, dans le problème ci-dessus, nous allons expliquer qu'il existe une contrainte pour chaque type de ressource et qu'elles ont toutes la même forme. Ainsi, les 7 contraintes seront résumées en une seule.

* **La formulation générique permet de séparer le modèle des données**. Ceci permet alors d'écrire une seule fois le modèle générique et de l'exécuter avec différentes données (la personne n'a même pas besoin de comprendre le modèle générique pour l'utiliser, il lui suffit de "saisir" les données). Ces données peuvent même se trouver dans des fichiers texte par exemple (une étape de lecture des des données pour initialiser les variables est alors nécessaire).



### Comment écrire une formulation générique ?

Nous montrons comment faire en expliquant la formulation générique sur le problème des yaourts. 

Numérotons les types de yaourts de 1 à 12 (Yaourt 1 est le yaourt à la fraise).
Les douze variables nécessaires peuvent être stockées dans un tableau de variables appelé `x`. Ainsi, `x[j]` va être la variable correspondant à la quantité fabriquée de yaourt de type `j` (`x[1]` représentera la quantité de yaourt à la fraise fabriqué).

Si les prix de vente sont stockés dans un tableau `v` tel que `v[j]` est le prix de vente au litre du yaourt de type `j`, alors la fonction objective est : 

`Max v[1] * x[1] + v[2] * x[2] + ... + v[12] * x[12]`

Mathématiquement, cela revient à écrire : $\sum_{j = 1}^{12} v[j]*x[j]$.
Il est possible d'écrire avec JuMP cette expression sous la forme :

`sum(v[j]*x[j] for j = 1:12)`


De la même manière, supposons que les quantités nécessaires pour produire chaque type de yaourt soient stockées dans une matrice `r` telle que `r[i,j]` représente la quantité de ressource `i` nécessaire pour produire un litre de yaourt de type `j`.
La contrainte liée à la quantité de fraise disponible est : 

`sum(r[1,j]*x[j] for j=1:12) <= 800`

On peut écrire une contrainte similaire pour chaque ressource de type `i`. Le modèle s'écrit alors sous forme générique de la manière suivante.

In [ ]:
#############
#  DONNÉES  #
#############

#Prix de vente de chaque type de ressource
v = [50, 48, 65, 47, 52, 46, 78, 75, 83, 59, 62, 76]

#Quantité de chaque ressource
q = [800, 200, 150, 80, 700, 500, 150]

#Quantités de ressources pour les différents types de yaourt
# R[i,j] : quantité de ressource i nécessaire pour fabriquer un litre de yaourt de type j
R = [
2  2  2  0  0  0  0  0  0  1  2  1;
0  0  0  3  2  2  0  0  0  2  1  1;
0  0  0  0  0  0  2  3  3  2  2  1;
0  0  2  5  0  2  0  0  3  1  0  3;
2  1  5  3  4  2  8  7  6  3  3  3;
1  0  1  4  0  2  2  0  2  4  2  1;
1  3  0  0  2  0  0  1  0  0  1  1;  
]


"""
    Résoud le problème de fabrication des yaourts

    Paramètres : 
    - R : matrice telle que R[i,j] indique la quantité de ressource numéro i 
      utilisée pour fabriquer un litre de yaourt de type j
    - v : prix de vente selon le type de yaourt
    - q : quantités initiales des différentes ressources

    Retourne l'optimum ainsi que la valeur de la solution optimale
"""
function solveYaourts(R, v, q)

    #Création d'un modèle. Ce modèle fera l'interface avec le solveur GLPK
    model = Model(with_optimizer(GLPK.Optimizer))
    
    #Nombre de variables
    n = length(v)
    
    #Nombre de ressources
    m = length(q)
    
    #On vérifie que les matrices et vecteurs ont les bonnes dimensions:
    # - le nombre de lignes de R doit être m
    @assert size(R,1) == m 
    
    # - le nombre de colonnes doit être n
    @assert size(R,2) == n

    #Création d'un tableau de n variables continues positives ou nulles.
    @variable(model, x[1:n] >= 0 )

    #Création de la fonction objective du modèle
    @objective(model, Max, sum(v[j] * x[j] for j=1:n) )

    #Ajout de 7 contraintes (une par ressource) dans le modèle
    for i in 1:m
        # La quantité de ressource numéro i utilisée pour fabriquer les yaourts ne dépasse pas
        # la quantité disponible
        @constraint(model, sum(R[i,j] * x[j] for j=1:n) <= q[i] )
    end

    #Écriture plus compacte :
    #@constraint(m,[i=1:m], sum(r[i,j] * x[j] for j=1:n) <= R[i] )

    #Affichage du modèle
    #print(m)

    #Résolution du problème d'optimisation linéaire m par le solveur GLPK
    optimize!(model)

    # Le problème doit être optimal (réalisable et borné)
    @assert termination_status(model) == OPTIMAL

    #value.(x) applique la fonction value à toutes les composantes de x
    return objective_value(model), value.(x)
end

optimum, sol = solveYaourts(R, v, q)

optimum = round(optimum; digits = 2)

sol = round.(sol; digits=3)

println("coût de la solution : ", optimum)
for i in 1:length(sol)
    if sol[i] >= 0.001
        println("sol[$i] = ", sol[i])
    end
end
println("Les autres variables valent 0")

### Questions

#### Question 1

Quelle quantité de chaque yaourt faut-il faire pour maximiser le prix de vente ?

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




#### Question 2

* Mauvaise surprise, 50kg de sucre sont envahis par des insectes et doivent donc être jetés. Cela modifie-t-il le plan de production ? Quelle est la perte associée ?

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




**Réponse :** Écrire votre réponse ici.




* Le problème est plus grave que prévu. En fait, 100kg de sucre (et non 50) doivent être jetés. Cela modifie-t-il le plan de production ? Quelle est la perte associée ?

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




**Réponse :** Écrire votre réponse ici.




#### Question 3

L'entreprise envisage aussi la possibilité de fabriquer un yaourt à la vanille. Celui-ci serait vendu 68 &euro; le litre. La fabrication d'un litre de ce yaourt nécessiterait 3kg de Vanille, 3 litres de lait et 3 litres de sucre. Est-ce intéressant de produire ce yaourt ?

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




**Réponse :** Écrire votre réponse ici.




## Exercice 1

Étienne commence sa préparation en vue de courrir un marathon. Pour se préparer, il doit suivre un régime particulier pour assurer ses besoins quotidiens en énergie (200kcal), protéïnes (55g) et lipides (800mg). Il peut sélectionner parmi les aliments suivants :



| Plats (portion)  | Énergie (kcal) | Protéïnes (g)| Lipides (mg) |
| -------          | -----          | ---          | ------------ |
| Pâtes bolognaise | 450            | 11           | 2            |
| Omelette fromage | 205            |  8           | 48           |
| Salade César     | 160            | 13           | 54           |
| Hamburger        | 230            | 28           | 285          |
| Curry légumes    | 420            |  6           | 320          |
| Couscous royal   | 360            | 18           | 40           |

Il peut manger plusieurs portions d'un même plat. De plus, les portions peuvent se consommer seulement en partie. 

Les portions ont un coût donné dans le tableau suivant :


| Plats (portion)  | Coût (en centimes) | 
| -------          | -----              |
| Pâtes bolognaise | 50                 | 
| Omelette fromage | 80                 | 
| Salade César     | 140                | 
| Hamburger        | 80                 | 
| Curry légumes    | 40                 | 
| Couscous royal   | 70                 | 



### Question 1

Déterminer un régime permettant à Étienne d'assurer ses besoins en énergie, protéïnes et lipides à un coût minimum.

La formulation que vous donnerez devra être générique.


In [ ]:
#############
#  DONNÉES  #
#############


plats = [
    "Pâtes bolognaise"
    "Omelette fromage"
    "Salade César"
    "Hamburger"
    "Curry légumes"
    "Couscous royal"
]

M = [
    250 105 80 72 130 180; 
     11   8 13 28   6  18; 
      2  48 54 88 320  40
]

prix = [50 80 140 80 40 70]
quantités = [2000, 55, 800];

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




### Question 2

Étienne se rend compte qu'il ne peut pas manger trop de portions d'un même plat sans être malade. Il a déterminé qu'il pouvait manger au plus :

| Plats            | Nombre de portions maximum | 
| -------          | -----                      |
| Pâtes bolognaise | 3                          | 
| Omelette fromage | 2                          | 
| Salade César     | 2                          | 
| Hamburger        | 3                          | 
| Curry légumes    | 3                          | 
| Couscous royal   | 2                          | 

Prendre en compte ce nombre maximum de portions dans le modèle.

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




## Exercice 2


Une entreprise d'électricité doit fournir de l’électricité pour les villes de Villetaneuse, Épinay et Saint-Denis. La puissance nécessaire d'électricité pour chaque ville est respectivement de 3000, 6000 et 5000 GW. Pour produire cette électricité, l’entreprise dispose de trois centrales électriques C1, C2 et C3. Le coût de production et d’acheminement de l’électricité en fonction des villes et des centrales (en &euro; par GW) est récapitulée dans le tableau suivant.



| Centrales | Villetaneuse | Épinay | Saint-Denis |
| ------- | -----        | ---    | ------------|
| C1      | 80           | 100    | 120         |
| C2      | 20           | 30     | 40          |
| C3      | 40           | 60     | 80          |


De plus, chaque centrale possède une puissance limitée. Cette limite est respectivement de 9000, 6000 et 7000 GW. L’entreprise souhaite déterminer la production d’électricité par centrale ainsi que la répartition de la production en fonction des trois villes de telle manière que le coût soit minimal. 



In [ ]:
#############
#  DONNÉES  #
#############


centrales = ["C1", "C2", "C3"]
puissances_max = [9000, 6000, 7000]

villes = ["Villetaneuse", "Épinay", "Saint-Denis"]
demandes = [3000  6000 5000]

coûts = [
    80 100 120;
    20  30  40;
    40  60  80
]

# Affichage des données

for i in 1:length(centrales)
    println("Centrale ", centrales[i], ", puissance maximum = ", puissances_max[i])
end

for j in 1:length(villes)
    println("Ville ", villes[j], ", demande = ", demandes[j])
end

for i in 1:3, j in 1:3
    println("Coût de ", centrales[i], " à ", villes[j], " = ", coûts[i,j])
end


### Question 1

Formuler ce problème sous forme générique.

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




### Question 2

Modifier uniquement les données en ajoutant une quatrième centrale de puissance maximum 3000 GW. Cette centrale peut délivrer l'électricité aux villes selon les coûts : 

|Villetaneuse | Épinay | Saint-Denis|
| -----        | ---    | ------------|
|10           | 20    | 15         |

Cela change-t-il la solution ?

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




## Exercice 2 : Epin&Drink&reg;

Une entreprise d'Épinay souhaite lancer en 2021 une nouvelle boisson nommée **Epin&Drink&reg;**. Le service commercial a réalisé une étude et ses prévisions de vente en hectolitres (hl) pour chaque mois de l'année 2021 sont données dans le tableau suivant :

|Mois | Janvier | Février | Mars | Avril | Mai | Juin | Juillet | Août | Septembre | Octobre | Novembre | Décembre|
|--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | ---|
| Volumes (hl) | 200 | 180 | 210 | 178 | 247 | 302 | 354 | 398 | 253 | 265 | 264 | 299|


Le coût de fabrication d'un hectolitre d'**Epin&Drink&reg;** varie tous les mois (suivant les équipes et les autres produits fabriqués par cette entreprise). Les estimations de ce coût sont donnés dans le tableau suivant :

| Janvier | Février | Mars | Avril | Mai | Juin | Juillet | Août | Septembre | Octobre | Novembre | Décembre|
|--- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- | --- |
| 54 | 63 | 32 | 55 | 96 | 24 | 46 | 94 | 116 | 87 | 75 | 73 |

Afin de satisfaire la demande et d'optimiser ses coûts de fabrication, l'entreprise peut stocker chaque mois une quantité de boisson pour la vendre plus tard (il n'y a aucun problème de date de péremption). Ce stockage coûte 40 &euro;/hectolitre/mois. La boisson fabriquée et vendue le même mois n'a pas besoin d'être stockée.

L'entreprise ayant déjà réalisé un test de production, elle possède 50 hectolitres de boisson au début du mois de janvier. De plus, elle peut au maximum produire 350 hectolitres et stocker 250 hectolitres chaque mois.

En tant qu'informaticien de cette entreprise, vous devez déterminer le plan de production de l'année 2021. Cette production doit permettre chaque mois de vendre le volume **Epin&Drink&reg;** prévu et le coût de cette production doit être minimum.
Quel est le coût total de fabrication/stockage sur l'année ?

In [ ]:
#############
#  DONNÉES  #
#############

#Volume des ventes
demandes = [200, 180, 210, 178, 247, 302, 354, 398, 253, 265, 264, 299]

c_prod = [54, 63, 32, 55, 96, 24, 46, 94, 116, 87, 75, 73]

c_stock = 40

s_deb = 50

prod_max = 350

stock_max = 250

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################




## Exercice 3 :  (issu du cours de Roland Grappe)

Une entreprise dispose de deux machines pour produire trois boissons. Les rythmes de production doivent être : 
* Epin&Drink&reg; : 42 litres par heure, 
* Epin&Drink&reg;Max&trade; : 49 litres par heure, 
* Villeta'up&trade; : 25 litres par heure.

Pour produire un litre de boisson, chaque machine met le temps suivant, en minutes :

|  -  | Machine 1 | Machine 2 |
|----|-----------|-----------|
| Epin&Drink&reg; | 5 | 6 |
| Epin&Drink&reg;Max&trade; | 3 |5|
| Villeta'up&trade; | 2 |

Les depenses de fabrication sont, par litre produit :

|  -  | Machine 1 | Machine 2 |
|----|-----------|-----------|
| Epin&Drink&reg; | 20 | 22 |
| Epin&Drink&reg;Max&trade; | 8 |12|
| Villeta'up&trade; | 12 |

### Questions

1. Formuler le problème qui consiste à répartir les productions sur les deux machines afin de minimiser les dépenses de production.
2. Écrire ce problème sous forme générique.
3. Modifer uniquement les données pour ajouter une troisième machine.

In [ ]:
#############
#  DONNÉES  #
#############


Boissons = ["Epin&drink", "Epin&drinkMax", "Villeta'up"]
Machines = ["M1", "M2"]

temps = [
    5 3 2 ;
    6 5 typemax(Int)
]

dépenses = [
    20  8 12 ;
    22 12 typemax(Int)
]

productions = [42, 49, 25];

In [ ]:
############################## 
#   Saisir votre code ici.   #
##############################


